In [29]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [30]:
files = os.listdir("./data")
files[:10]

['2018-12-05.html',
 '2018-12-06.html',
 '2018-12-07(1).html',
 '2018-12-07(2).html',
 '2018-12-07.html',
 '2018-12-08.html',
 '2018-12-09.html',
 '2018-12-10.html',
 '2018-12-14.html',
 '2018-12-15.html']

In [31]:
date_pattern = re.compile(r'\d{4}-\d{2}-\d{2}')
dates = [date_pattern.search(filename).group(0) for filename in files]
dates[:10]

['2018-12-05',
 '2018-12-06',
 '2018-12-07',
 '2018-12-07',
 '2018-12-07',
 '2018-12-08',
 '2018-12-09',
 '2018-12-10',
 '2018-12-14',
 '2018-12-15']

In [32]:
reports = []
for i, file in enumerate(files):
    with open(f'data/{file}', 'r', encoding='utf-8') as file:
        html = file.read()

    soup = BeautifulSoup(html)
    main_content = soup.find('main', {'id': 'content'})

    # Extract any list content that is not related to the report
    unwanted_list_elements = main_content.find_all(['span', 'a', 'i'])
    for element in unwanted_list_elements:
        element.extract()

    content = main_content.find_all(['p', 'h1', 'li'])


    text = " ".join(element.get_text() for element in content if element.get_text())

    # Clean the report text
    text = text.replace('\xa0', ' ')
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)

    reports.append(text)
    print(f"Appending {i+1}/{len(files)}", end='\r')
reports[:10]

['Acompaña presidente Andrés Manuel López Obrador a Claudia Sheinbaum Pardo a rendir protesta como Jefa de Gobierno El presidente Andrés Manuel López Obrador acompañó a la doctora Claudia Sheinbaum Pardo en la ceremonia de la toma de protesta como Jefa de Gobierno de la Ciudad de México. “Me da mucho gusto porque Claudia Sheinbaum es una mujer con convicciones, es una mujer inteligente, es una mujer honesta y va a hacer un buen gobierno”, expresó durante la conferencia de prensa matutina. En el salón Tesorería de Palacio Nacional informó que unirán esfuerzos el gobierno federal y capitalino en materia de seguridad, por lo que todos los lunes la primera jefa de Gobierno estará presente en las reuniones del gabinete de seguridad que se realizan a las 6:00 horas. “Estamos trabajando en forma coordinada (…) los lunes porque ella tiene que atender todos los días los asuntos de la ciudad. La invitamos y va a asistir para ponernos de acuerdo sobre los temas de seguridad en la Ciudad de México

In [33]:
bulletins = {
    'Date' : dates,
    'Report' : reports,
}

bulletins_df = pd.DataFrame(bulletins)
bulletins_df

,Date,Report
0,2018-12-05,Acompaña presidente Andrés Manuel López Obrado...
1,2018-12-06,Revela presidente Andrés Manuel López Obrador ...
2,2018-12-07,"En Tuxpan, presidente de México anuncia plan d..."
3,2018-12-07,Destaca presidente avances en sistema de infor...
4,2018-12-07,"En Acaponeta, presidente López Obrador anuncia..."
...,...,...
2317,2023-04-12,Departamento de Justicia de Estados Unidos dev...
2318,2023-04-12,Países atenderán tráfico de drogas y control d...
2319,2023-04-13,"Nuevo Laredo será sede de la Aduana Nacional, ..."
2320,2023-04-13,Presidente recibe visita de Jeremy Corbyn en P...


In [34]:
# Save the DataFrame to a CSV file
bulletins_df.to_csv('bulletins.csv', index=False)